In [1]:
!pip install ttach

You are using pip version 9.0.1, however version 21.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install git+https://github.com/ildoonet/cutmix

  Cloning https://github.com/ildoonet/cutmix to /tmp/pip-e3hlxm45-build
You are using pip version 9.0.1, however version 21.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install resnest --pre

You are using pip version 9.0.1, however version 21.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# 导入各种包
import torch
import torch.nn as nn
from torch.nn import functional as F
import ttach as tta
from resnest.torch import resnest50

from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
# This is for the progress bar.
from tqdm import tqdm

In [5]:
import os
import moxing as mox


file_name = 'classify-leaves'
if not os.path.exists(file_name):
    mox.file.copy_parallel('obs://test444/input/' + file_name, file_name)

labels_dataframe = pd.read_csv('classify-leaves/train.csv')
labels_dataframe.head(5)

INFO:root:Using MoXing-v2.0.0.rc0-19e4d3ab
INFO:root:Using OBS-Python-SDK-3.20.9.1
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 8000
INFO:root:Listing OBS: 9000
INFO:root:Listing OBS: 10000
INFO:root:Listing OBS: 11000
INFO:root:Listing OBS: 12000
INFO:root:Listing OBS: 13000
INFO:root:Listing OBS: 14000
INFO:root:Listing OBS: 15000
INFO:root:Listing OBS: 16000
INFO:root:Listing OBS: 17000
INFO:root:Listing OBS: 18000
INFO:root:Listing OBS: 19000
INFO:root:Listing OBS: 20000
INFO:root:Listing OBS: 21000
INFO:root:Listing OBS: 22000
INFO:root:Listing OBS: 23000
INFO:root:Listing OBS: 24000
INFO:root:Listing OBS: 25000
INFO:root:Listing OBS: 26000
INFO:root:Listing OBS: 27000
INFO:root:pid: None.	1000/27156
INFO:root:pid: None.	2000/27156
INFO:root:pid: None.	3000/27156
INFO:root:pid: None.	4000/27156
INFO:root:pid: 

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [6]:
# 把label文件排个序
leaves_labels = sorted(list(set(labels_dataframe['label'])))
n_classes = len(leaves_labels)
print(n_classes)
leaves_labels[:10]

176


['abies_concolor',
 'abies_nordmanniana',
 'acer_campestre',
 'acer_ginnala',
 'acer_griseum',
 'acer_negundo',
 'acer_palmatum',
 'acer_pensylvanicum',
 'acer_platanoides',
 'acer_pseudoplatanus']

In [7]:
# 把label转成对应的数字
class_to_num = dict(zip(leaves_labels, range(n_classes)))
class_to_num

{'abies_concolor': 0,
 'abies_nordmanniana': 1,
 'acer_campestre': 2,
 'acer_ginnala': 3,
 'acer_griseum': 4,
 'acer_negundo': 5,
 'acer_palmatum': 6,
 'acer_pensylvanicum': 7,
 'acer_platanoides': 8,
 'acer_pseudoplatanus': 9,
 'acer_rubrum': 10,
 'acer_saccharinum': 11,
 'acer_saccharum': 12,
 'aesculus_flava': 13,
 'aesculus_glabra': 14,
 'aesculus_hippocastamon': 15,
 'aesculus_pavi': 16,
 'ailanthus_altissima': 17,
 'albizia_julibrissin': 18,
 'amelanchier_arborea': 19,
 'amelanchier_canadensis': 20,
 'amelanchier_laevis': 21,
 'asimina_triloba': 22,
 'betula_alleghaniensis': 23,
 'betula_jacqemontii': 24,
 'betula_lenta': 25,
 'betula_nigra': 26,
 'betula_populifolia': 27,
 'broussonettia_papyrifera': 28,
 'carpinus_betulus': 29,
 'carpinus_caroliniana': 30,
 'carya_cordiformis': 31,
 'carya_glabra': 32,
 'carya_ovata': 33,
 'carya_tomentosa': 34,
 'castanea_dentata': 35,
 'catalpa_bignonioides': 36,
 'catalpa_speciosa': 37,
 'cedrus_atlantica': 38,
 'cedrus_deodara': 39,
 'cedru

In [8]:
# 再转换回来，方便最后预测的时候使用
num_to_class = {v : k for k, v in class_to_num.items()}
num_to_class

{0: 'abies_concolor',
 1: 'abies_nordmanniana',
 2: 'acer_campestre',
 3: 'acer_ginnala',
 4: 'acer_griseum',
 5: 'acer_negundo',
 6: 'acer_palmatum',
 7: 'acer_pensylvanicum',
 8: 'acer_platanoides',
 9: 'acer_pseudoplatanus',
 10: 'acer_rubrum',
 11: 'acer_saccharinum',
 12: 'acer_saccharum',
 13: 'aesculus_flava',
 14: 'aesculus_glabra',
 15: 'aesculus_hippocastamon',
 16: 'aesculus_pavi',
 17: 'ailanthus_altissima',
 18: 'albizia_julibrissin',
 19: 'amelanchier_arborea',
 20: 'amelanchier_canadensis',
 21: 'amelanchier_laevis',
 22: 'asimina_triloba',
 23: 'betula_alleghaniensis',
 24: 'betula_jacqemontii',
 25: 'betula_lenta',
 26: 'betula_nigra',
 27: 'betula_populifolia',
 28: 'broussonettia_papyrifera',
 29: 'carpinus_betulus',
 30: 'carpinus_caroliniana',
 31: 'carya_cordiformis',
 32: 'carya_glabra',
 33: 'carya_ovata',
 34: 'carya_tomentosa',
 35: 'castanea_dentata',
 36: 'catalpa_bignonioides',
 37: 'catalpa_speciosa',
 38: 'cedrus_atlantica',
 39: 'cedrus_deodara',
 40: 'c

In [9]:
# 继承pytorch的dataset，创建自己的
class TrainValidData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform=None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式
        self.transform = transform

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 第二列是图像的 label
        self.label_arr = np.asarray(self.data_info.iloc[1:,1])
        # 计算 length
        self.data_len = len(self.data_info.index) - 1

    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)

        # 得到图像的 label
        label = self.label_arr[index]
        number_label = class_to_num[label]

        return (img_as_img, number_label)  #返回每一个index对应的图片数据和对应的label

    def __len__(self):
        return self.data_len

In [10]:
# 继承pytorch的dataset，创建自己的
class TestData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform = None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.transform = transform
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 计算 length
        self.data_len = len(self.data_info.index) - 1
        
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)


        return img_as_img

    def __len__(self):
        return self.data_len

In [11]:
train_transform = transforms.Compose([
    # 随机裁剪图像，所得图像为原始面积的0.08到1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224 x 224的新图像
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    # 添加随机噪声
    transforms.ToTensor(),
    # 标准化图像的每个通道
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
val_test_transform = transforms.Compose([
    transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [12]:
train_val_path = 'classify-leaves/train.csv'
test_path = 'classify-leaves/test.csv'
# csv文件中已经images的路径了，因此这里只到上一级目录
img_path = 'classify-leaves/'

train_val_dataset = TrainValidData(train_val_path, img_path)
test_dataset = TestData(test_path, img_path, transform = val_test_transform)
print(train_val_dataset.data_info)
print(test_dataset.data_info)

                      0                             1
0                 image                         label
1          images/0.jpg              maclura_pomifera
2          images/1.jpg              maclura_pomifera
3          images/2.jpg              maclura_pomifera
4          images/3.jpg              maclura_pomifera
5          images/4.jpg              maclura_pomifera
6          images/5.jpg              maclura_pomifera
7          images/6.jpg                   ulmus_rubra
8          images/7.jpg      broussonettia_papyrifera
9          images/8.jpg              maclura_pomifera
10         images/9.jpg      broussonettia_papyrifera
11        images/10.jpg              maclura_pomifera
12        images/11.jpg             prunus_virginiana
13        images/12.jpg              maclura_pomifera
14        images/13.jpg             prunus_virginiana
15        images/14.jpg      broussonettia_papyrifera
16        images/15.jpg              maclura_pomifera
17        images/16.jpg     

In [13]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# ResNeSt模型
def resnest_model(num_classes, feature_extract = False):
    model_ft = resnest50(pretrained=True)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [14]:
# 看一下是在cpu还是GPU上
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [15]:
# Configuration options
k_folds = 2
num_epochs = 1
learning_rate = 1e-4
weight_decay = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
valid_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [16]:
import moxing as mox
import os
if not os.path.exists('/home/ma-user/.cache/torch/checkpoints/resnest50-528c19ca.pth'):
    mox.file.copy('obs://test444/resnest50-528c19ca.pth', '/home/ma-user/.cache/torch/checkpoints/resnest50-528c19ca.pth')

In [17]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, valid_ids) in enumerate(kfold.split(train_val_dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
        CutMix(TrainValidData(train_val_path, img_path, transform=train_transform), num_class=176, beta=1.0, prob=0.5,
               num_mix=2),
        batch_size=32, sampler=train_subsampler, num_workers=0)
    validloader = torch.utils.data.DataLoader(
        TrainValidData(train_val_path, img_path, transform=val_test_transform),
        batch_size=32, sampler=valid_subsampler, num_workers=0)

    # Initialize a model and put it on the device specified.
    model = resnest_model(176)
    model = model.to(device)
    model.device = device

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=10)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):
        model.train()
        # Print epoch
        print(f'Starting epoch {epoch + 1}')
        # These are used to record information in training
        train_losses = []
        train_accs = []
        # Iterate the training set by batches
        for batch in tqdm(trainloader):
            # Move images and labels to GPU
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)
            # Forward the data
            logits = model(imgs)
            # Calculate loss
            loss = train_loss_function(logits, labels)
            # Clear gradients in previous step
            optimizer.zero_grad()
            # Compute gradients for parameters
            loss.backward()
            # Update the parameters with computed gradients
            optimizer.step()
            # Compute the accuracy for current batch.
            # acc = (logits.argmax(dim=-1) == labels).float().mean()
            # Record the loss and accuracy.
            train_losses.append(loss.item())
            # train_accs.append(acc)
        print("第%d个epoch的学习率：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))
        scheduler.step()
        # The average loss and accuracy of the training set is the average of the recorded values.
        train_loss = np.sum(train_losses) / len(train_losses)
        # train_acc = np.sum(train_accs) / len(train_accs)
        # Print the information.
        # print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}")

    # Train process (all epochs) is complete
    print('Training process has finished. Saving trained model.')
    print('Starting validation')

    # Saving the model
    print('saving model with loss {:.3f}'.format(train_loss))
    save_path = f'./model-fold-{fold}.pth'
    torch.save(model.state_dict(), save_path)
    # Start Validation
    model.eval()
    valid_losses = []
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(validloader):
            imgs, labels = batch
            # No gradient in validation
            logits = model(imgs.to(device))
            loss = valid_loss_function(logits, labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            # Record loss and accuracy
            valid_losses.append(loss.item())
            valid_accs.append(acc)
        # The average loss and accuracy
        valid_loss = np.sum(valid_losses) / len(valid_losses)
        valid_acc = np.sum(valid_accs) / len(valid_accs)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        print('Accuracy for fold %d: %d' % (fold, valid_acc))
        print('--------------------------------------')
        results[fold] = valid_acc
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation += value
print(f'Average: {total_summation / len(results.items())} ')

--------------------------------------
FOLD 0
--------------------------------------


  0%|          | 0/287 [00:00<?, ?it/s]

Starting epoch 1


  0%|          | 0/287 [00:00<?, ?it/s]

第1个epoch的学习率：0.000100
[ Train | 001/001 ] loss = 3.73038
Training process has finished. Saving trained model.
Starting validation
saving model with loss 3.730


...

Starting epoch 1


  0%|          | 0/287 [00:00<?, ?it/s]

第1个epoch的学习率：0.000100
[ Train | 001/001 ] loss = 3.66106
Training process has finished. Saving trained model.
Starting validation
saving model with loss 3.661


100%|██████████| 287/287 [00:36<00:00,  7.90it/s]

[ Valid | 001/001 ] loss = 1.27000, acc = 0.66322
Accuracy for fold 1: 0
--------------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 2 FOLDS
--------------------------------
Fold 0: 0.6656054258346558 
Fold 1: 0.6632186770439148 
Average: 0.6644120216369629 


In [18]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = val_test_transform),
                      batch_size=32, num_workers=0)

In [19]:
## predict
model = resnest_model(176)

# create model and load weights from checkpoint
model = model.to(device)
# load the all folds
for test_fold in range(k_folds):
    model_path = f'./model-fold-{test_fold}.pth'
    saveFileName = f'./submission-fold-{test_fold}.csv'
    model.load_state_dict(torch.load(model_path))

    # Make sure the model is in eval mode.
    # Some modules like Dropout or BatchNorm affect if the model is in training mode.
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200, 200))  # Test-Time Augmentation

    # Initialize a list to store the predictions.
    predictions = []
    # Iterate the testing set by batches.
    for batch in tqdm(testloader):
        imgs = batch
        with torch.no_grad():
            logits = tta_model(imgs.to(device))

        # Take the class with greatest logit as prediction and record it.
        predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

    preds = []
    for i in predictions:
        preds.append(num_to_class[i])

    test_data = pd.read_csv(test_path)
    test_data['label'] = pd.Series(preds)
    submission = pd.concat([test_data['image'], test_data['label']], axis=1)
    submission.to_csv(saveFileName, index=False)
    print("ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

  0%|          | 0/275 [00:00<?, ?it/s]

ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [01:49<00:00,  2.50it/s]

ResNeSt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [20]:
# 读取5折交叉验证的结果
df0 = pd.read_csv('./submission-fold-0.csv')
df1 = pd.read_csv('./submission-fold-1.csv')
# df2 = pd.read_csv('./submission-fold-2.csv')
# df3 = pd.read_csv('./submission-fold-3.csv')
# df4 = pd.read_csv('./submission-fold-4.csv')

In [21]:
# 往第0折结果里添加数字化标签列
list_num_label0 = []
for i in df0['label']:
    list_num_label0.append(class_to_num[i])
df0['num_label0']=list_num_label0
df0.head()

,image,label,num_label0
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,populus_grandidentata,123
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_cembra,103
4,images/18357.jpg,platanus_acerifolia,120


In [22]:
# 往第1折结果里添加数字化标签列
list_num_label1 = []
for i in df1['label']:
    list_num_label1.append(class_to_num[i])
df1['num_label1']=list_num_label1
df1.head()

,image,label,num_label1
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,platanus_acerifolia,120
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_rigida,113
4,images/18357.jpg,platanus_occidentalis,121


In [23]:
# # 往第2折结果里添加数字化标签列
# list_num_label2 = []
# for i in df2['label']:
#     list_num_label2.append(class_to_num[i])
# df2['num_label2']=list_num_label2
# df2.head()

In [24]:
# # 往第3折结果里添加数字化标签列
# list_num_label3 = []
# for i in df3['label']:
#     list_num_label3.append(class_to_num[i])
# df3['num_label3']=list_num_label3
# df3.head()

In [25]:
# # 往第4折结果里添加数字化标签列
# list_num_label4 = []
# for i in df4['label']:
#     list_num_label4.append(class_to_num[i])
# df4['num_label4']=list_num_label4
# df4.head()

In [26]:
# 准备整合5折的结果到同一个DataFrame
df_all = df0.copy()
df_all.drop(['label'],axis=1,inplace=True)
df_all.head()

,image,num_label0
0,images/18353.jpg,22
1,images/18354.jpg,123
2,images/18355.jpg,120
3,images/18356.jpg,103
4,images/18357.jpg,120


In [27]:
# 整合5折的数字化标签结果到同一个DataFrame
df_all['num_label1']=list_num_label1
# df_all['num_label2']=list_num_label2
# df_all['num_label3']=list_num_label3
# df_all['num_label4']=list_num_label4
df_all.head()

,image,num_label0,num_label1
0,images/18353.jpg,22,22
1,images/18354.jpg,123,120
2,images/18355.jpg,120,120
3,images/18356.jpg,103,113
4,images/18357.jpg,120,121


In [28]:
# 对df_all进行转置，方便求众数
df_all_transpose = df_all.copy().drop(['image'],axis=1).transpose()
df_all_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,8790,8791,8792,8793,8794,8795,8796,8797,8798,8799
num_label0,22,123,120,103,120,74,52,91,165,169,...,117,53,117,57,70,119,19,117,46,73
num_label1,22,120,120,113,121,74,48,91,165,169,...,112,53,108,57,169,112,70,112,169,73


In [29]:
# 求得投票众数
df_mode = df_all_transpose.mode().transpose()
df_mode.head()

,0,1
0,22.0,NaN
1,120.0,123.0
2,120.0,NaN
3,103.0,113.0
4,120.0,121.0


In [30]:
# 把投票结果的数字化标签转成字符串标签
voting_class = []
for each in df_mode[0]:
    voting_class.append(num_to_class[each])
voting_class

['asimina_triloba',
 'platanus_acerifolia',
 'platanus_acerifolia',
 'pinus_cembra',
 'platanus_acerifolia',
 'koelreuteria_paniculata',
 'chionanthus_virginicus',
 'metasequoia_glyptostroboides',
 'tilia_cordata',
 'ulmus_americana',
 'quercus_marilandica',
 'ilex_opaca',
 'asimina_triloba',
 'castanea_dentata',
 'abies_concolor',
 'metasequoia_glyptostroboides',
 'abies_concolor',
 'castanea_dentata',
 'pinus_cembra',
 'cryptomeria_japonica',
 'ailanthus_altissima',
 'morus_alba',
 'cercis_canadensis',
 'ptelea_trifoliata',
 'fagus_grandifolia',
 'liriodendron_tulipifera',
 'acer_rubrum',
 'pinus_bungeana',
 'quercus_montana',
 'cornus_mas',
 'magnolia_tripetala',
 'ficus_carica',
 'maclura_pomifera',
 'quercus_palustris',
 'crataegus_viridis',
 'ilex_opaca',
 'magnolia_macrophylla',
 'acer_palmatum',
 'abies_concolor',
 'abies_concolor',
 'ilex_opaca',
 'liriodendron_tulipifera',
 'cornus_mas',
 'pinus_nigra',
 'gleditsia_triacanthos',
 'malus_floribunda',
 'carya_glabra',
 'acer_ps

In [31]:
# 将投票结果的字符串标签添加到df_all中
df_all['label'] = voting_class
df_all.head()

,image,num_label0,num_label1,label
0,images/18353.jpg,22,22,asimina_triloba
1,images/18354.jpg,123,120,platanus_acerifolia
2,images/18355.jpg,120,120,platanus_acerifolia
3,images/18356.jpg,103,113,pinus_cembra
4,images/18357.jpg,120,121,platanus_acerifolia


In [32]:
# 提取image和label两列为最终的结果
df_submission = df_all[['image','label']].copy()
df_submission.head()

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,platanus_acerifolia
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_cembra
4,images/18357.jpg,platanus_acerifolia


In [33]:
# 保存当前模型得到的最终结果
df_submission.to_csv('./submission-resnest.csv', index=False)
print('Voting results of resnest successfully saved!')

Voting results of resnest successfully saved!


In [34]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# resnext50_32x4d模型
def resnext_model(num_classes, feature_extract = False, use_pretrained=True):

    model_ft = models.resnext50_32x4d(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [35]:
# Configuration options
k_folds = 2
num_epochs = 1
learning_rate = 1e-3
weight_decay = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
valid_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [36]:
if not os.path.exists('/home/ma-user/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth'):
    mox.file.copy('obs://test444/resnext50_32x4d-7cdf4587.pth', '/home/ma-user/.cache/torch/checkpoints/resnext50_32x4d-7cdf4587.pth')

In [37]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, valid_ids) in enumerate(kfold.split(train_val_dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
        CutMix(TrainValidData(train_val_path, img_path, transform=train_transform), num_class=176, beta=1.0, prob=0.5,
               num_mix=2),
        batch_size=128, sampler=train_subsampler, num_workers=0)
    validloader = torch.utils.data.DataLoader(
        TrainValidData(train_val_path, img_path, transform=val_test_transform),
        batch_size=128, sampler=valid_subsampler, num_workers=0)

    # Initialize a model and put it on the device specified.
    model = resnext_model(176)
    model = model.to(device)
    model.device = device

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    #   optimizer = SWA(our_optimizer, swa_start=5, swa_freq =5, swa_lr=0.05)
    scheduler = CosineAnnealingLR(optimizer, T_max=10)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):
        model.train()
        # Print epoch
        print(f'Starting epoch {epoch + 1}')
        # These are used to record information in training
        train_losses = []
        train_accs = []
        # Iterate the training set by batches
        for batch in tqdm(trainloader):
            # Move images and labels to GPU
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)
            # Forward the data
            logits = model(imgs)
            # Calculate loss
            loss = train_loss_function(logits, labels)
            # Clear gradients in previous step
            optimizer.zero_grad()
            # Compute gradients for parameters
            loss.backward()
            # Update the parameters with computed gradients
            optimizer.step()
            # Compute the accuracy for current batch.
            # acc = (logits.argmax(dim=-1) == labels).float().mean()
            # Record the loss and accuracy.
            train_losses.append(loss.item())
            # train_accs.append(acc)
        print("第%d个epoch的学习率：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))
        scheduler.step()
        # The average loss and accuracy of the training set is the average of the recorded values.
        train_loss = np.sum(train_losses) / len(train_losses)
        # train_acc = np.sum(train_accs) / len(train_accs)
        # Print the information.
        # print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}")

    # Train process (all epochs) is complete
    print('Training process has finished. Saving trained model.')
    print('Starting validation')

    # Saving the model
    print('saving model with loss {:.3f}'.format(train_loss))
    save_path = f'./model-fold-{fold}.pth'
    torch.save(model.state_dict(), save_path)
    # Start Validation
    model.eval()
    valid_losses = []
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(validloader):
            imgs, labels = batch
            # No gradient in validation
            logits = model(imgs.to(device))
            loss = valid_loss_function(logits, labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            # Record loss and accuracy
            valid_losses.append(loss.item())
            valid_accs.append(acc)
        # The average loss and accuracy
        valid_loss = np.sum(valid_losses) / len(valid_losses)
        valid_acc = np.sum(valid_accs) / len(valid_accs)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        print('Accuracy for fold %d: %d' % (fold, valid_acc))
        print('--------------------------------------')
        results[fold] = valid_acc
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation += value
print(f'Average: {total_summation / len(results.items())} ')

--------------------------------------
FOLD 0
--------------------------------------


  0%|          | 0/72 [00:00<?, ?it/s]

Starting epoch 1


  0%|          | 0/72 [00:00<?, ?it/s]

第1个epoch的学习率：0.001000
[ Train | 001/001 ] loss = 3.86847
Training process has finished. Saving trained model.
Starting validation
saving model with loss 3.868


...

Starting epoch 1


  0%|          | 0/72 [00:00<?, ?it/s]

第1个epoch的学习率：0.001000
[ Train | 001/001 ] loss = 3.76916
Training process has finished. Saving trained model.
Starting validation
saving model with loss 3.769


100%|██████████| 72/72 [00:33<00:00,  2.16it/s]

[ Valid | 001/001 ] loss = 2.60917, acc = 0.33620
Accuracy for fold 1: 0
--------------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 2 FOLDS
--------------------------------
Fold 0: 0.4088907539844513 
Fold 1: 0.33620384335517883 
Average: 0.37254729866981506 


In [38]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = val_test_transform),
                      batch_size=128, num_workers=0)

In [39]:
## predict
model = resnext_model(176)

# create model and load weights from checkpoint
model = model.to(device)
# load the all folds
for test_fold in range(k_folds):
    model_path = f'./model-fold-{test_fold}.pth'
    saveFileName = f'./submission-fold-{test_fold}.csv'
    model.load_state_dict(torch.load(model_path))

    # Make sure the model is in eval mode.
    # Some modules like Dropout or BatchNorm affect if the model is in training mode.
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200, 200))

    # Initialize a list to store the predictions.
    predictions = []
    # Iterate the testing set by batches.
    for batch in tqdm(testloader):
        imgs = batch
        with torch.no_grad():
            logits = tta_model(imgs.to(device))

        # Take the class with greatest logit as prediction and record it.
        predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

    preds = []
    for i in predictions:
        preds.append(num_to_class[i])

    test_data = pd.read_csv(test_path)
    test_data['label'] = pd.Series(preds)
    submission = pd.concat([test_data['image'], test_data['label']], axis=1)
    submission.to_csv(saveFileName, index=False)
    print("ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

  0%|          | 0/69 [00:00<?, ?it/s]

ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:38<00:00,  1.42s/it]

ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [40]:
df0 = pd.read_csv('./submission-fold-0.csv')
df1 = pd.read_csv('./submission-fold-1.csv')
# df2 = pd.read_csv('./submission-fold-2.csv')
# df3 = pd.read_csv('./submission-fold-3.csv')
# df4 = pd.read_csv('./submission-fold-4.csv')

In [41]:
list_num_label0 = []
for i in df0['label']:
    list_num_label0.append(class_to_num[i])
df0['num_label0']=list_num_label0
df0.head()

,image,label,num_label0
0,images/18353.jpg,magnolia_macrophylla,82
1,images/18354.jpg,quercus_michauxii,143
2,images/18355.jpg,quercus_muehlenbergii,145
3,images/18356.jpg,pinus_koraiensis,107
4,images/18357.jpg,paulownia_tomentosa,97


In [42]:
list_num_label1 = []
for i in df1['label']:
    list_num_label1.append(class_to_num[i])
df1['num_label1']=list_num_label1
df1.head()

,image,label,num_label1
0,images/18353.jpg,magnolia_acuminata,79
1,images/18354.jpg,quercus_shumardii,150
2,images/18355.jpg,ginkgo_biloba,66
3,images/18356.jpg,pinus_pungens,111
4,images/18357.jpg,quercus_shumardii,150


In [43]:
# list_num_label2 = []
# for i in df2['label']:
#     list_num_label2.append(class_to_num[i])
# df2['num_label2']=list_num_label2
# df2.head()

In [44]:
# list_num_label3 = []
# for i in df3['label']:
#     list_num_label3.append(class_to_num[i])
# df3['num_label3']=list_num_label3
# df3.head()

In [45]:
# list_num_label4 = []
# for i in df4['label']:
#     list_num_label4.append(class_to_num[i])
# df4['num_label4']=list_num_label4
# df4.head()

In [46]:
df_all = df0.copy()
df_all.drop(['label'],axis=1,inplace=True)
df_all.head()

,image,num_label0
0,images/18353.jpg,82
1,images/18354.jpg,143
2,images/18355.jpg,145
3,images/18356.jpg,107
4,images/18357.jpg,97


In [47]:
df_all['num_label1']=list_num_label1
# df_all['num_label2']=list_num_label2
# df_all['num_label3']=list_num_label3
# df_all['num_label4']=list_num_label4
df_all.head()

,image,num_label0,num_label1
0,images/18353.jpg,82,79
1,images/18354.jpg,143,150
2,images/18355.jpg,145,66
3,images/18356.jpg,107,111
4,images/18357.jpg,97,150


In [48]:
df_all_transpose = df_all.copy().drop(['image'],axis=1).transpose()
df_all_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,8790,8791,8792,8793,8794,8795,8796,8797,8798,8799
num_label0,82,143,145,107,97,74,4,91,86,95,...,107,174,107,27,73,107,20,111,73,46
num_label1,79,150,66,111,150,74,129,91,134,95,...,117,57,117,129,73,117,57,117,73,46


In [49]:
df_mode = df_all_transpose.mode().transpose()
df_mode.head()

,0,1
0,79.0,82.0
1,143.0,150.0
2,66.0,145.0
3,107.0,111.0
4,97.0,150.0


In [50]:
voting_class = []
for each in df_mode[0]:
    voting_class.append(num_to_class[each])
voting_class

['magnolia_acuminata',
 'quercus_michauxii',
 'ginkgo_biloba',
 'pinus_koraiensis',
 'paulownia_tomentosa',
 'koelreuteria_paniculata',
 'acer_griseum',
 'metasequoia_glyptostroboides',
 'malus_baccata',
 'ostrya_virginiana',
 'ginkgo_biloba',
 'quercus_muehlenbergii',
 'magnolia_grandiflora',
 'castanea_dentata',
 'abies_concolor',
 'metasequoia_glyptostroboides',
 'prunus_virginiana',
 'castanea_dentata',
 'pinus_cembra',
 'cryptomeria_japonica',
 'carya_cordiformis',
 'malus_baccata',
 'cercis_canadensis',
 'ptelea_trifoliata',
 'halesia_tetraptera',
 'cercis_canadensis',
 'acer_rubrum',
 'pinus_koraiensis',
 'castanea_dentata',
 'cornus_kousa',
 'acer_pensylvanicum',
 'catalpa_bignonioides',
 'carya_cordiformis',
 'quercus_coccinea',
 'crataegus_viridis',
 'cercis_canadensis',
 'cornus_florida',
 'acer_griseum',
 'picea_orientalis',
 'larix_decidua',
 'quercus_virginiana',
 'cercis_canadensis',
 'malus_baccata',
 'pinus_koraiensis',
 'gleditsia_triacanthos',
 'malus_baccata',
 'aes

In [51]:
df_all['label'] = voting_class
df_all.head()

,image,num_label0,num_label1,label
0,images/18353.jpg,82,79,magnolia_acuminata
1,images/18354.jpg,143,150,quercus_michauxii
2,images/18355.jpg,145,66,ginkgo_biloba
3,images/18356.jpg,107,111,pinus_koraiensis
4,images/18357.jpg,97,150,paulownia_tomentosa


In [52]:
df_submission = df_all[['image','label']].copy()
df_submission.head()

,image,label
0,images/18353.jpg,magnolia_acuminata
1,images/18354.jpg,quercus_michauxii
2,images/18355.jpg,ginkgo_biloba
3,images/18356.jpg,pinus_koraiensis
4,images/18357.jpg,paulownia_tomentosa


In [53]:
df_submission.to_csv('./submission-resnext.csv', index=False)
print('ResNeXt voting results successfully saved!')

ResNeXt voting results successfully saved!


In [54]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# densenet161模型
def dense_model(num_classes, feature_extract = False, use_pretrained=True):

    model_ft = models.densenet161(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [55]:
# Configuration options
k_folds = 2
num_epochs = 1
learning_rate = 1e-4
weight_decay = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
valid_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

In [56]:
if not os.path.exists('/home/ma-user/.cache/torch/checkpoints/densenet161-8d451a50.pth'):
    mox.file.copy('obs://test444/densenet161-8d451a50.pth', '/home/ma-user/.cache/torch/checkpoints/densenet161-8d451a50.pth')

In [57]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, valid_ids) in enumerate(kfold.split(train_val_dataset)):

    # Print
    print(f'FOLD {fold}')
    print('--------------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
        CutMix(TrainValidData(train_val_path, img_path, transform=train_transform), num_class=176, beta=1.0, prob=0.5,
               num_mix=2),
        batch_size=32, sampler=train_subsampler, num_workers=0)
    validloader = torch.utils.data.DataLoader(
        TrainValidData(train_val_path, img_path, transform=val_test_transform),
        batch_size=32, sampler=valid_subsampler, num_workers=0)

    # Initialize a model and put it on the device specified.
    model = dense_model(176)
    model = model.to(device)
    model.device = device

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=10)

    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):
        model.train()
        # Print epoch
        print(f'Starting epoch {epoch + 1}')
        # These are used to record information in training
        train_losses = []
        train_accs = []
        # Iterate the training set by batches
        for batch in tqdm(trainloader):
            # Move images and labels to GPU
            imgs, labels = batch
            imgs = imgs.to(device)
            labels = labels.to(device)
            # Forward the data
            logits = model(imgs)
            # Calculate loss
            loss = train_loss_function(logits, labels)
            # Clear gradients in previous step
            optimizer.zero_grad()
            # Compute gradients for parameters
            loss.backward()
            # Update the parameters with computed gradients
            optimizer.step()
            # Compute the accuracy for current batch.
            #       acc = (logits.argmax(dim=-1) == labels).float().mean()
            # Record the loss and accuracy.
            train_losses.append(loss.item())
        #       train_accs.append(acc)
        print("第%d个epoch的学习率：%f" % (epoch + 1, optimizer.param_groups[0]['lr']))
        scheduler.step()
        # The average loss and accuracy of the training set is the average of the recorded values.
        train_loss = np.sum(train_losses) / len(train_losses)
        #     train_acc = np.sum(train_accs) / len(train_accs)
        # Print the information.
        #     print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}")

    # Train process (all epochs) is complete
    print('Training process has finished. Saving trained model.')
    print('Starting validation')

    # Saving the model
    print('saving model with loss {:.3f}'.format(train_loss))
    save_path = f'./model-fold-{fold}.pth'
    torch.save(model.state_dict(), save_path)
    # Start Validation
    model.eval()
    valid_losses = []
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(validloader):
            imgs, labels = batch
            # No gradient in validation
            logits = model(imgs.to(device))
            loss = valid_loss_function(logits, labels.to(device))
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            # Record loss and accuracy
            valid_losses.append(loss.item())
            valid_accs.append(acc)
        # The average loss and accuracy
        valid_loss = np.sum(valid_losses) / len(valid_losses)
        valid_acc = np.sum(valid_accs) / len(valid_accs)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        print('Accuracy for fold %d: %d' % (fold, valid_acc))
        print('--------------------------------------')
        results[fold] = valid_acc
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} ')
    total_summation += value
print(f'Average: {total_summation / len(results.items())} ')

--------------------------------------
FOLD 0
--------------------------------------


  0%|          | 0/287 [00:00<?, ?it/s]

Starting epoch 1


  0%|          | 0/287 [00:00<?, ?it/s]

第1个epoch的学习率：0.000100
[ Train | 001/001 ] loss = 4.04629
Training process has finished. Saving trained model.
Starting validation
saving model with loss 4.046


...

Starting epoch 1


100%|██████████| 287/287 [02:41<00:00,  1.78it/s]


第1个epoch的学习率：0.000100
[ Train | 001/001 ] loss = 4.07571
Training process has finished. Saving trained model.
Starting validation
saving model with loss 4.076


100%|██████████| 287/287 [00:52<00:00,  5.52it/s]

[ Valid | 001/001 ] loss = 1.95003, acc = 0.57756
Accuracy for fold 1: 0
--------------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 2 FOLDS
--------------------------------
Fold 0: 0.5573127269744873 
Fold 1: 0.5775624513626099 
Average: 0.5674375891685486 


In [58]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = val_test_transform),
                      batch_size=32, num_workers=0)

In [59]:
## predict
model = dense_model(176)

# create model and load weights from checkpoint
model = model.to(device)
# load the all folds
for test_fold in range(k_folds):
    model_path = f'./model-fold-{test_fold}.pth'
    saveFileName = f'./submission-fold-{test_fold}.csv'
    model.load_state_dict(torch.load(model_path))

    # Make sure the model is in eval mode.
    # Some modules like Dropout or BatchNorm affect if the model is in training mode.
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200, 200))

    # Initialize a list to store the predictions.
    predictions = []
    # Iterate the testing set by batches.
    for batch in tqdm(testloader):
        imgs = batch
        with torch.no_grad():
            logits = tta_model(imgs.to(device))

        # Take the class with greatest logit as prediction and record it.
        predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

    preds = []
    for i in predictions:
        preds.append(num_to_class[i])

    test_data = pd.read_csv(test_path)
    test_data['label'] = pd.Series(preds)
    submission = pd.concat([test_data['image'], test_data['label']], axis=1)
    submission.to_csv(saveFileName, index=False)
    print("Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

100%|██████████| 275/275 [02:43<00:00,  1.68it/s]


Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 275/275 [02:43<00:00,  1.68it/s]

Dense Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [60]:
df0 = pd.read_csv('./submission-fold-0.csv')
df1 = pd.read_csv('./submission-fold-1.csv')
# df2 = pd.read_csv('./submission-fold-2.csv')
# df3 = pd.read_csv('./submission-fold-3.csv')
# df4 = pd.read_csv('./submission-fold-4.csv')

In [61]:
list_num_label0 = []
for i in df0['label']:
    list_num_label0.append(class_to_num[i])
df0['num_label0']=list_num_label0
df0.head()

,image,label,num_label0
0,images/18353.jpg,maclura_pomifera,78
1,images/18354.jpg,populus_grandidentata,123
2,images/18355.jpg,quercus_velutina,152
3,images/18356.jpg,pinus_cembra,103
4,images/18357.jpg,quercus_velutina,152


In [62]:
list_num_label1 = []
for i in df1['label']:
    list_num_label1.append(class_to_num[i])
df1['num_label1']=list_num_label1
df1.head()

,image,label,num_label1
0,images/18353.jpg,diospyros_virginiana,59
1,images/18354.jpg,acer_pseudoplatanus,9
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,acer_saccharum,12


In [63]:
# list_num_label2 = []
# for i in df2['label']:
#     list_num_label2.append(class_to_num[i])
# df2['num_label2']=list_num_label2
# df2.head()

In [64]:
# list_num_label3 = []
# for i in df3['label']:
#     list_num_label3.append(class_to_num[i])
# df3['num_label3']=list_num_label3
# df3.head()

In [65]:
# list_num_label4 = []
# for i in df4['label']:
#     list_num_label4.append(class_to_num[i])
# df4['num_label4']=list_num_label4
# df4.head()

In [66]:
df_all = df0.copy()
df_all.drop(['label'],axis=1,inplace=True)
df_all.head()

,image,num_label0
0,images/18353.jpg,78
1,images/18354.jpg,123
2,images/18355.jpg,152
3,images/18356.jpg,103
4,images/18357.jpg,152


In [67]:
df_all['num_label1']=list_num_label1
# df_all['num_label2']=list_num_label2
# df_all['num_label3']=list_num_label3
# df_all['num_label4']=list_num_label4
df_all.head()

,image,num_label0,num_label1
0,images/18353.jpg,78,59
1,images/18354.jpg,123,9
2,images/18355.jpg,152,120
3,images/18356.jpg,103,102
4,images/18357.jpg,152,12


In [68]:
df_all_transpose = df_all.copy().drop(['image'],axis=1).transpose()
df_all_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,8790,8791,8792,8793,8794,8795,8796,8797,8798,8799
num_label0,78,123,152,103,152,74,78,91,165,169,...,116,170,116,90,78,116,59,116,45,45
num_label1,59,9,120,102,12,74,22,91,86,30,...,112,57,112,57,109,119,162,112,46,45


In [69]:
df_mode = df_all_transpose.mode().transpose()
df_mode.head()

,0,1
0,59.0,78.0
1,9.0,123.0
2,120.0,152.0
3,102.0,103.0
4,12.0,152.0


In [70]:
voting_class = []
for each in df_mode[0]:
    voting_class.append(num_to_class[each])
voting_class

['diospyros_virginiana',
 'acer_pseudoplatanus',
 'platanus_acerifolia',
 'pinus_bungeana',
 'acer_saccharum',
 'koelreuteria_paniculata',
 'asimina_triloba',
 'metasequoia_glyptostroboides',
 'malus_baccata',
 'carpinus_caroliniana',
 'quercus_marilandica',
 'prunus_sargentii',
 'asimina_triloba',
 'castanea_dentata',
 'abies_concolor',
 'metasequoia_glyptostroboides',
 'prunus_virginiana',
 'quercus_michauxii',
 'pinus_cembra',
 'cryptomeria_japonica',
 'fraxinus_nigra',
 'broussonettia_papyrifera',
 'cercis_canadensis',
 'ptelea_trifoliata',
 'cornus_florida',
 'liriodendron_tulipifera',
 'acer_rubrum',
 'pinus_cembra',
 'quercus_montana',
 'cornus_florida',
 'magnolia_macrophylla',
 'broussonettia_papyrifera',
 'staphylea_trifolia',
 'quercus_palustris',
 'prunus_virginiana',
 'maclura_pomifera',
 'asimina_triloba',
 'acer_palmatum',
 'tsuga_canadensis',
 'abies_concolor',
 'quercus_virginiana',
 'liriodendron_tulipifera',
 'magnolia_stellata',
 'pinus_pungens',
 'gleditsia_triacan

In [71]:
df_all['label'] = voting_class
df_all.head()

,image,num_label0,num_label1,label
0,images/18353.jpg,78,59,diospyros_virginiana
1,images/18354.jpg,123,9,acer_pseudoplatanus
2,images/18355.jpg,152,120,platanus_acerifolia
3,images/18356.jpg,103,102,pinus_bungeana
4,images/18357.jpg,152,12,acer_saccharum


In [72]:
df_submission = df_all[['image','label']].copy()
df_submission.head()

,image,label
0,images/18353.jpg,diospyros_virginiana
1,images/18354.jpg,acer_pseudoplatanus
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,acer_saccharum


In [73]:
df_submission.to_csv('./submission-densenet.csv', index=False)
print('Densenet results successfully saved!')

Densenet results successfully saved!


In [74]:
df_resnest = pd.read_csv('submission-resnest.csv')
df_resnext = pd.read_csv('submission-resnext.csv')
df_densenet = pd.read_csv('submission-densenet.csv')

In [75]:
df_all = df_resnest.copy()
df_all.rename(columns = {'label':'label_resnest'},inplace=True)
df_all['label_resnext'] = df_resnext.copy()['label']
df_all['label_densenet'] = df_densenet.copy()['label']
df_all.head()

,image,label_resnest,label_resnext,label_densenet
0,images/18353.jpg,asimina_triloba,magnolia_acuminata,diospyros_virginiana
1,images/18354.jpg,platanus_acerifolia,quercus_michauxii,acer_pseudoplatanus
2,images/18355.jpg,platanus_acerifolia,ginkgo_biloba,platanus_acerifolia
3,images/18356.jpg,pinus_cembra,pinus_koraiensis,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia,paulownia_tomentosa,acer_saccharum


In [76]:
df_all['label']=0
for rows in range(len(df_all)):
    if (df_all['label_resnest'].iloc[rows]==df_all['label_resnext'].iloc[rows]) or (df_all['label_resnest'].iloc[rows]==df_all['label_densenet'].iloc[rows]):
        df_all['label'].iloc[rows] = df_all.copy()['label_resnest'].iloc[rows]
    elif df_all['label_resnext'].iloc[rows]==df_all['label_densenet'].iloc[rows]:
        df_all['label'].iloc[rows] = df_all.copy()['label_resnext'].iloc[rows]
    else:
        df_all['label'].iloc[rows] = df_all.copy()['label_resnest'].iloc[rows]
df_all.head()

/home/ma-user/anaconda3/envs/Pytorch-1.4.0/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,image,label_resnest,label_resnext,label_densenet,label
0,images/18353.jpg,asimina_triloba,magnolia_acuminata,diospyros_virginiana,asimina_triloba
1,images/18354.jpg,platanus_acerifolia,quercus_michauxii,acer_pseudoplatanus,platanus_acerifolia
2,images/18355.jpg,platanus_acerifolia,ginkgo_biloba,platanus_acerifolia,platanus_acerifolia
3,images/18356.jpg,pinus_cembra,pinus_koraiensis,pinus_bungeana,pinus_cembra
4,images/18357.jpg,platanus_acerifolia,paulownia_tomentosa,acer_saccharum,platanus_acerifolia


In [77]:
df_final = df_all.copy()[['image','label']]
df_final.head()

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,platanus_acerifolia
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_cembra
4,images/18357.jpg,platanus_acerifolia


In [78]:
df_final.to_csv('./submission.csv', index=False)
print('Final results successfully saved!')

Final results successfully saved!
